In [9]:
import numpy as np
# import polars as pl

import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

from bokeh.models import ColumnDataSource

output_notebook()

Loading BokehJS ...

The `ColumnDataSource` is a mapping of column names (strings) to sequences of values. Example: The mapping is provided by passing a dict with string keys and simple lists as values. The values could also be numpy arrays or pandas sequences.
It is the central data source object used throughout bokeh. When a Bokeh widget is constructed, it is passed a data source object as a parameter. The widget will then display data from the columns of the data source. The widget will also update whenever the data source is changed.

> **NOTE**: All the columns in a `ColumnDataSource` must always be the _SAME_ length.


The example below will be simply to illustrate a surface plot using `ColumnDataSource` and `surface3d` from `bokeh.plotting` module. It will be a volatility surface of a stock option, especially showing Vanna.


In [10]:
def _vanna(S, K, T, r, sigma):
    lyst = []
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * T**1 / 2
    return (
        (1 / np.sqrt(2 * np.pi) * S * np.exp(-(d1**2) * 1 / 2) * np.sqrt(T)) /
        S * (1 - d1 / (sigma * np.sqrt(T))))

In [11]:
K = 100
T = 1
r = 0

underlying_1d = np.linspace(100, 120, 25)
sigma_1d = np.linspace(0.05, 0.3, 50)

print(underlying_1d)
print(f"------------------------------------------\n")
print(sigma_1d)

[100.         100.83333333 101.66666667 102.5        103.33333333
 104.16666667 105.         105.83333333 106.66666667 107.5
 108.33333333 109.16666667 110.         110.83333333 111.66666667
 112.5        113.33333333 114.16666667 115.         115.83333333
 116.66666667 117.5        118.33333333 119.16666667 120.        ]
------------------------------------------

[0.05       0.05510204 0.06020408 0.06530612 0.07040816 0.0755102
 0.08061224 0.08571429 0.09081633 0.09591837 0.10102041 0.10612245
 0.11122449 0.11632653 0.12142857 0.12653061 0.13163265 0.13673469
 0.14183673 0.14693878 0.15204082 0.15714286 0.1622449  0.16734694
 0.17244898 0.17755102 0.18265306 0.1877551  0.19285714 0.19795918
 0.20306122 0.20816327 0.21326531 0.21836735 0.22346939 0.22857143
 0.23367347 0.23877551 0.24387755 0.24897959 0.25408163 0.25918367
 0.26428571 0.26938776 0.2744898  0.27959184 0.28469388 0.28979592
 0.29489796 0.3       ]


In [12]:
underlying, sigma = np.meshgrid(underlying_1d, sigma_1d)

underlying, sigma

(array([[100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ],
        [100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ],
        [100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ],
        ...,
        [100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ],
        [100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ],
        [100.        , 100.83333333, 101.66666667, ..., 118.33333333,
         119.16666667, 120.        ]]),
 array([[0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
         0.05      ],
        [0.05510204, 0.05510204, 0.05510204, ..., 0.05510204, 0.05510204,
         0.05510204],
        [0.06020408, 0.06020408, 0.06020408, ..., 0.06020408, 0.06020408,
         0.06020408],
        ...,
        [0.28979592, 0.28979592, 0.2

In [13]:
# vanna_data = _vanna(underlying, K, T, r, sigma)
vanna_df = pd.DataFrame(_vanna(underlying, K, T, r, sigma),
                        columns=underlying_1d,
                        index=sigma_1d)

# vanna_df = pl.DataFrame(vanna_data,
#                         columns=underlying_1d,
#                         index=sigma_1d)
vanna_df

,100.000000,100.833333,101.666667,102.500000,103.333333,104.166667,105.000000,105.833333,106.666667,107.500000,...,112.500000,113.333333,114.166667,115.000000,115.833333,116.666667,117.500000,118.333333,119.166667,120.000000
0.050000,0.199409,-1.104500,-2.288849,-3.269779,-3.991950,-4.432137,-4.597635,-4.520560,-4.249702,-3.841707,...,-1.093270,-0.809203,-0.585411,-0.414308,-0.287074,-0.194891,-0.129721,-0.084707,-0.054296,-0.034183
0.055102,0.199395,-0.876912,-1.869371,-2.719265,-3.384904,-3.844145,-4.094219,-4.149272,-4.036324,-3.790464,...,-1.465940,-1.156141,-0.894175,-0.678743,-0.506032,-0.370790,-0.267189,-0.189448,-0.132242,-0.090921
0.060204,0.199381,-0.703957,-1.546407,-2.285774,-2.890509,-3.341436,-3.631991,-3.767167,-3.761458,-3.636217,...,-1.774621,-1.468518,-1.194703,-0.956266,-0.753582,-0.585032,-0.447680,-0.337842,-0.251548,-0.184875
0.065306,0.199365,-0.569496,-1.292821,-1.939577,-2.485667,-2.915099,-3.220307,-3.401711,-3.466655,-3.427920,...,-2.004496,-1.721786,-1.456808,-1.215034,-0.999572,-0.811571,-0.650653,-0.515331,-0.403388,-0.312200
0.070408,0.199348,-0.462924,-1.090270,-1.659399,-2.151697,-2.554023,-2.859012,-3.064902,-3.174975,-3.196710,...,-2.157675,-1.908926,-1.666115,-1.435600,-1.221905,-1.027899,-0.855034,-0.703600,-0.572996,-0.461977
0.075510,0.199329,-0.377046,-0.926041,-1.429859,-1.873952,-2.247571,-2.544019,-2.760597,-2.898308,-2.961357,...,-2.244505,-2.033588,-1.819907,-1.609785,-1.408225,-1.218947,-1.044492,-0.886367,-0.745201,-0.620919
0.080612,0.199309,-0.306840,-0.791113,-1.239691,-1.641071,-1.986458,-2.269956,-2.488571,-2.642054,-2.732598,...,-2.277930,-2.104475,-1.922302,-1.737220,-1.554130,-1.377000,-1.208888,-1.051999,-0.907772,-0.776985
0.085714,0.199288,-0.248720,-0.678954,-1.080530,-1.444254,-1.762904,-2.031373,-2.246697,-2.407967,-2.516150,...,-2.270506,-2.131789,-1.980585,-1.821986,-1.660491,-1.499948,-1.343533,-1.193763,-1.052530,-0.921154
0.090816,0.199266,-0.200067,-0.584744,-0.946081,-1.276656,-1.570507,-1.823249,-2.032098,-2.195830,-2.314670,...,-2.233117,-2.125321,-2.002895,-1.870173,-1.731124,-1.589280,-1.447696,-1.308938,-1.175083,-1.047745
0.095918,0.199242,-0.158936,-0.504868,-0.831548,-1.132923,-1.404047,-1.641164,-1.841737,-2.004424,-2.129008,...,-2.174619,-2.093611,-1.996980,-1.888351,-1.771133,-1.648459,-1.523136,-1.397616,-1.273986,-1.153971


In [14]:
vanna_df.index.name = "volatility"
vanna_df.columns.name = "Strike"
vanna_df.iloc[:10, :10]

Strike,100.000000,100.833333,101.666667,102.500000,103.333333,104.166667,105.000000,105.833333,106.666667,107.500000
volatility,,,,,,,,,,
0.050000,0.199409,-1.104500,-2.288849,-3.269779,-3.991950,-4.432137,-4.597635,-4.520560,-4.249702,-3.841707
0.055102,0.199395,-0.876912,-1.869371,-2.719265,-3.384904,-3.844145,-4.094219,-4.149272,-4.036324,-3.790464
0.060204,0.199381,-0.703957,-1.546407,-2.285774,-2.890509,-3.341436,-3.631991,-3.767167,-3.761458,-3.636217
0.065306,0.199365,-0.569496,-1.292821,-1.939577,-2.485667,-2.915099,-3.220307,-3.401711,-3.466655,-3.427920
0.070408,0.199348,-0.462924,-1.090270,-1.659399,-2.151697,-2.554023,-2.859012,-3.064902,-3.174975,-3.196710
0.075510,0.199329,-0.377046,-0.926041,-1.429859,-1.873952,-2.247571,-2.544019,-2.760597,-2.898308,-2.961357
0.080612,0.199309,-0.306840,-0.791113,-1.239691,-1.641071,-1.986458,-2.269956,-2.488571,-2.642054,-2.732598
0.085714,0.199288,-0.248720,-0.678954,-1.080530,-1.444254,-1.762904,-2.031373,-2.246697,-2.407967,-2.516150
0.090816,0.199266,-0.200067,-0.584744,-0.946081,-1.276656,-1.570507,-1.823249,-2.032098,-2.195830,-2.314670


In [15]:
from bokeh.palettes import Sunset10

_x, _y, _z = underlying_1d, sigma_1d, vanna_df.values

_fig = figure(width=700, height=500)

lvls = np.linspace(-10.0, 10.0, 110)

contour_renderer = _fig.contour(x=_x,
                                y=_y,
                                z=_z,
                                levels=lvls,
                                fill_color=Sunset10,
                                line_color="black")
colorbar = contour_renderer.construct_color_bar(title="Vanna")
_fig.add_layout(colorbar, "right")

show(_fig)
# fig = contour(, x=underlying_1d, y=sigma_1d, z=vanna_df.values)

In [16]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Surface(z=vanna_df.values, x=underlying_1d, y=sigma_1d))
fig.update_traces(contours_z=dict(
    show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))
fig.update_layout(title="Vanna", autosize=False, width=900, height=700)
fig.show()